In [6]:
# @title Dependencies
import torch
import torchvision
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"device: {device}")

device: cuda


In [15]:
# @title Model definition { display-mode: "form" }

# model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=False)
# Repacing last 1k classes linear layer with 2 classes layer
# model.classifier[-1] = torch.nn.Linear(in_features=4096, out_features=2)

from torch import nn

class ClassifierModel(nn.Module):
  def __init__(self):
    super(ClassifierModel, self).__init__()
    self.features = nn.Sequential(
        nn.Conv2d(3,16, kernel_size=(2,2)), # 512x512x3xN --> 510x510x64xN, rf = 3x3 32-> 16
        nn.MaxPool2d(kernel_size=2, stride=2), # 510x510x64xN --> 255x255x64xN
        nn.ReLU(),
        nn.Conv2d(16,128, kernel_size=(2,2)),  # 255x255x64xN --> 252x252x128xN, rf = 6x6 16-> 8
        nn.AvgPool2d(kernel_size=2, stride=2), # 252x252x128xN --> 126x126x128xN
        nn.ReLU(),
        # nn.Conv2d(128,64, kernel_size=(3,3)), # 126x126x128xN --> 124x124x64xN, rf = 8x8
        # nn.MaxPool2d(kernel_size=2, stride=2), # 124x124x64xN --> 62x62x64xN
        # nn.ReLU(),
        # nn.Conv2d(64,16, kernel_size=(3,3)), # 62x62x64xN --> 61x61x16xN, rf = 10x10
        # nn.MaxPool2d(kernel_size=2, stride=2), # 60x60x16xN --> 30x30x16xN
        # nn.ReLU(),
    )
    self.converter = nn.Flatten() # 30x30x16xN --> 14400 params
    self.classifier = nn.Sequential(
        nn.Linear(in_features=307328, out_features=4096),
        nn.Sigmoid(),
        nn.Linear(in_features=4096, out_features=2048),
        nn.Sigmoid(),
        nn.Linear(in_features=2048, out_features=512),
        nn.Sigmoid(),
        nn.Linear(in_features=512, out_features=128),
        nn.Sigmoid(),
        nn.Linear(in_features=128, out_features=32),
        nn.Sigmoid(),
        nn.Linear(in_features=32, out_features=8),
        nn.Sigmoid(),
        nn.Linear(in_features=8, out_features=2)
    )

  def forward(self, x):
    x = self.features(x)
    x = self.converter(x)
    x = self.classifier(x)
    return x

In [ ]:
# @title Loading data { display-mode: "form" }

from torchvision.transforms import v2
from torch import Tensor

preprocess = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True)
])
    #t    ransforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

torch.manual_seed(0)
dataset = torchvision.datasets.ImageFolder(
    root=r'C:\\Users\\bes-s\\OneDrive\\Документы\\NN\\datasets\\CIFAKE\\train',
    transform=preprocess
    )

trainsplit = 0.9
# More batch, because 50 000 files in each folder
batch_size = 64

dataset_size = len(dataset)
print(f"Size of dataset is: {dataset_size}, train/test split ratio: {trainsplit}, batch size: {batch_size}")
trainset, testset = torch.utils.data.random_split(dataset, [trainsplit, 1-trainsplit])

train_loader = torch.utils.data.DataLoader(dataset=trainset, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(dataset=testset, batch_size=batch_size)

In [8]:
from datasets import load_dataset
from transformers import AutoImageProcessor
dataset = load_dataset("mvkvc/artifact-100k") # subset of ArtiFact labled with "ai" and "real" tags

dataset = dataset.with_format(type="torch", device=device)

image_processor = AutoImageProcessor.from_pretrained("umm-maybe/AI-image-detector")

batch_size = 40

train_loader = torch.utils.data.DataLoader(dataset=dataset["train"], batch_size=batch_size,  num_workers=2)
test_loader = torch.utils.data.DataLoader(dataset=dataset["test"], batch_size=batch_size, num_workers=2)

In [18]:
import torch.optim as optim

model = ClassifierModel()
model.to(device)

loss_func = torch.nn.CrossEntropyLoss()
optimizer = optim.ASGD(model.parameters())

In [19]:
# @title Training loop { display-mode: "form" }

import torch
import datetime
from torch.utils.tensorboard import SummaryWriter

def calculate_epoch(writer: SummaryWriter, epoch, is_train=False):
  if is_train:
    model.train(True)
  else:
    model.eval()

  total = 0
  correct = 0
  running_loss = 0
  for batch_ind, data in enumerate(train_loader):
    inputs, targets = data['image'], data['label']
    inputs = inputs.permute(0,3,1,2)
    inputs, targets = inputs.to(torch.float32).to(device), targets.to(torch.long).to(device)
    targets = targets.unsqueeze(1)
    optimizer.zero_grad()
    output = model(inputs)

    total += len(output)
    correct += (output.argmax() == targets).float().sum()

    targets = torch.nn.functional.one_hot(targets, num_classes = 2).to(torch.float)
    loss = loss_func(output, targets)
    if is_train:
      loss.backward()
      optimizer.step()

    running_loss += loss.item()

    if batch_ind % 10 == 9:
      stage = "train" if is_train else "test"
      avg_loss = running_loss / 10
      writer.add_scalar(f"loss/{stage}", avg_loss, epoch*batch_size + batch_ind)
      writer.add_scalar(f"accuracy/{stage}", correct/total, epoch*batch_size + batch_ind)
      running_loss = 0


def train_loop():
  time_now = datetime.datetime.now()
  layout = {
      "Model statistics" : {
      "loss": ["Multiline", ["loss/train", "loss/test"]],
      "accuracy": ["Multiline", ["accuracy/train", "accuracy/test"]]
      }
  }
  writer = SummaryWriter(log_dir=time_now.strftime("C:\\Users\\bes-s\\OneDrive\\Документы\\NN\\runs\\Alex-Artifact\\run %d-%m %H_%M_%S"))
  writer.add_custom_scalars(layout)
  epoch_count = 100
  for epoch in range(epoch_count):  # loop over the dataset multiple times
    running_loss = 0.0
    print(f"epoch #{epoch+1}/{epoch_count}...", end = "")
    calculate_epoch(writer, epoch, is_train=True)
    with torch.no_grad():
      calculate_epoch(writer, epoch, is_train=False)
    print("done.")
    writer.flush()

  writer.close()
  print('done training.')


train_loop()

epoch #1/100...

RuntimeError: 0D or 1D target tensor expected, multi-target not supported

In [ ]:
# move the input and model to GPU for speed if available
# if torch.cuda.is_available():
#     # testset = testset.cuda()
#     model.to('cuda')

model.eval()
with torch.no_grad():
    for inputs, targets in test_loader:
      output = model(inputs)
      output = torch.argmax(output, dim=1)
      print(f"in: {targets}, out: {output}")
# Tensor of shape 1000, with confidence scores over ImageNet's 1000 classes
print(output[0])
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
probabilities = torch.nn.functional.softmax(output[0], dim=0)
print(probabilities)